In [1]:
# Importamos librerías

import numpy as np
import pandas as pd
from pathlib import Path

# Paths del proyecto
PROJECT_ROOT = Path(".").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
ARTIFACTS = PROJECT_ROOT / "artifacts"
ARTIFACTS.mkdir(parents=True, exist_ok=True)

data_path = DATA_PROCESSED / "dataset_monthly.csv.gz"
print("Loading:", data_path)

full = pd.read_csv(data_path)
print("full shape:", full.shape)
full.head()

Loading: /Users/andrespadronquintana/Desktop/METODOS_GRAN_ESCALA/TAREAS/tarea1_future_sales/data/processed/dataset_monthly.csv.gz
full shape: (18129480, 11)


,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,item_category_id,item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag6,item_cnt_month_lag12
0,0,0,32,6.0,221.0,40,0.0,0.0,0.0,0.0,0.0
1,0,0,33,3.0,347.0,37,0.0,0.0,0.0,0.0,0.0
2,0,0,35,1.0,247.0,40,0.0,0.0,0.0,0.0,0.0
3,0,0,43,1.0,221.0,40,0.0,0.0,0.0,0.0,0.0
4,0,0,51,2.0,128.5,57,0.0,0.0,0.0,0.0,0.0


In [2]:
TARGET = "item_cnt_month"

# Features (todas menos el target)
feature_cols = [
    "date_block_num",
    "shop_id",
    "item_id",
    "item_price_mean",
    "item_category_id",
    "item_cnt_month_lag1",
    "item_cnt_month_lag2",
    "item_cnt_month_lag3",
    "item_cnt_month_lag6",
    "item_cnt_month_lag12",
]

# Split temporal
train_df = full[full["date_block_num"] <= 32].copy()
valid_df = full[full["date_block_num"] == 33].copy()
test_df = full[full["date_block_num"] == 34].copy()

X_train, y_train = train_df[feature_cols], train_df[TARGET]
X_valid, y_valid = valid_df[feature_cols], valid_df[TARGET]
X_test = test_df[feature_cols]

print("Train:", X_train.shape, "Valid:", X_valid.shape, "Test:", X_test.shape)

Train: (17388360, 10) Valid: (526920, 10) Test: (214200, 10)


In [3]:
cat_cols = ["shop_id", "item_id", "item_category_id"]

In [4]:
import sklearn

sklearn.__version__

'1.8.0'

In [7]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error([1, 2], [1, 2])

0.0

In [8]:
from sklearn.metrics import root_mean_squared_error

pred_valid_lgb = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)

rmse_lgb = root_mean_squared_error(y_valid, pred_valid_lgb)
print("LightGBM VALID RMSE (mes 33, todo):", rmse_lgb)
print("Best iteration:", model_lgb.best_iteration)

NameError: name 'model_lgb' is not defined

In [9]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lgb_train = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols, free_raw_data=False)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols, free_raw_data=False)

params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "min_data_in_leaf": 50,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "lambda_l2": 1.0,
    "verbosity": -1,
    "seed": 42,
}

model_lgb = lgb.train(
    params,
    lgb_train,
    num_boost_round=3000,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=["train", "valid"],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(50)],
)

pred_valid_lgb = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
rmse_lgb = mean_squared_error(y_valid, pred_valid_lgb, squared=False)

print("LightGBM VALID RMSE (mes 33, todo):", rmse_lgb)
print("Best iteration:", model_lgb.best_iteration)

Training until validation scores don't improve for 100 rounds
[50]	train's rmse: 0.5362	valid's rmse: 0.511337
[100]	train's rmse: 0.487969	valid's rmse: 0.510292
[150]	train's rmse: 0.471757	valid's rmse: 0.50824
[200]	train's rmse: 0.463058	valid's rmse: 0.506381
[250]	train's rmse: 0.45646	valid's rmse: 0.506055
[300]	train's rmse: 0.4516	valid's rmse: 0.505635
[350]	train's rmse: 0.447399	valid's rmse: 0.505441
[400]	train's rmse: 0.443476	valid's rmse: 0.505056
[450]	train's rmse: 0.44039	valid's rmse: 0.504746
[500]	train's rmse: 0.437143	valid's rmse: 0.504549
[550]	train's rmse: 0.434439	valid's rmse: 0.50446
[600]	train's rmse: 0.431716	valid's rmse: 0.504311
[650]	train's rmse: 0.428918	valid's rmse: 0.504329
[700]	train's rmse: 0.426284	valid's rmse: 0.504023
[750]	train's rmse: 0.423956	valid's rmse: 0.503866
[800]	train's rmse: 0.421893	valid's rmse: 0.503618
[850]	train's rmse: 0.419975	valid's rmse: 0.50348
[900]	train's rmse: 0.417942	valid's rmse: 0.503443
[950]	train'

TypeError: got an unexpected keyword argument 'squared'

In [11]:
pred_valid_lgb = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
rmse_lgb = np.sqrt(np.mean((y_valid.values - pred_valid_lgb) ** 2))
print("LightGBM VALID RMSE (mes 33):", rmse_lgb)

LightGBM VALID RMSE (mes 33): 0.4981650718343212


In [12]:
X_trainval = pd.concat([X_train, X_valid], ignore_index=True)
y_trainval = pd.concat([y_train, y_valid], ignore_index=True)

lgb_trainval = lgb.Dataset(
    X_trainval, label=y_trainval, categorical_feature=cat_cols, free_raw_data=False
)

model_lgb_final = lgb.train(params, lgb_trainval, num_boost_round=model_lgb.best_iteration)

In [13]:
pred_test = model_lgb_final.predict(X_test, num_iteration=model_lgb_final.current_iteration())
pred_test = np.clip(pred_test, 0, 20)  # por consistencia con el target clip
pred_test[:10]

array([0.        , 0.        , 0.3477753 , 0.        , 0.        ,
       0.07000902, 0.        , 0.00311263, 0.        , 0.        ])

In [18]:
# 1) Cargar test ORIGINAL (el que trae ID)
test_raw_path = PROJECT_ROOT / "data" / "raw" / "test.csv"  # ajusta si tu ruta cambia
test_raw = pd.read_csv(test_raw_path)

# limpiar nombres por si hay espacios raros
test_raw.columns = test_raw.columns.str.strip()

# si viene como 'id' en vez de 'ID'
if "ID" not in test_raw.columns and "id" in test_raw.columns:
    test_raw = test_raw.rename(columns={"id": "ID"})

print(test_raw.columns)
assert set(["ID", "shop_id", "item_id"]).issubset(test_raw.columns), (
    "Tu test_raw NO es el original de Kaggle"
)

# 2) Crear df de predicciones con llaves (shop_id, item_id)
pred_df = test_df[["shop_id", "item_id"]].copy()
pred_df["item_cnt_month"] = pred_test  # pred_test debe corresponder a test_df en el mismo orden
pred_df["item_cnt_month"] = pred_df["item_cnt_month"].clip(0, 20)

# 3) Merge sobre test_raw para respetar el orden Kaggle
sub = test_raw.merge(pred_df, on=["shop_id", "item_id"], how="left", validate="one_to_one")


sub["item_cnt_month"] = sub["item_cnt_month"].fillna(0).clip(0, 20)

submission = sub[["ID", "item_cnt_month"]]

# 4) Checks finales
assert len(submission) == len(test_raw)
assert submission["ID"].notna().all()

sub_path = ARTIFACTS / "submission_lgb.csv"
submission.to_csv(sub_path, index=False)
print("Saved:", sub_path)
submission.head()

Index(['ID', 'shop_id', 'item_id'], dtype='object')
Saved: /Users/andrespadronquintana/Desktop/METODOS_GRAN_ESCALA/TAREAS/tarea1_future_sales/artifacts/submission_lgb.csv


,ID,item_cnt_month
0,0,0.000000
1,1,0.000000
2,2,0.347775
3,3,0.000000
4,4,0.000000
